In [ ]:
from google.colab import files
files.upload()

In [ ]:
!cat inatur_urls.csv

In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

In [ ]:
df = pd.read_csv('inatur_urls.csv')
df

,image_url,scientific_name,common_name
0,https://inaturalist-open-data.s3.amazonaws.com...,Gloeoporus dichrous,Глеопорус двухцветный
1,https://inaturalist-open-data.s3.amazonaws.com...,Cortinarius caperatus,Колпак кольчатый
2,https://inaturalist-open-data.s3.amazonaws.com...,Cantharellus cibarius,Лисичка обыкновенная
3,https://inaturalist-open-data.s3.amazonaws.com...,Fomitopsis pinicola,Трутовик окаймлённый
4,https://inaturalist-open-data.s3.amazonaws.com...,Fomitopsis betulina,Трутовик берёзовый
...,...,...,...
12265,https://inaturalist-open-data.s3.amazonaws.com...,Cladonia deformis,Кладония бесформенная
12266,https://inaturalist-open-data.s3.amazonaws.com...,Hypocenomyce scalaris,Гипоценомице ступенчатый
12267,https://inaturalist-open-data.s3.amazonaws.com...,Fomitopsis pinicola,Трутовик окаймлённый
12268,https://inaturalist-open-data.s3.amazonaws.com...,Hypogymnia physodes,Гипогимния вздутая


In [ ]:
#example of the url
df.image_url[12269]

'https://inaturalist-open-data.s3.amazonaws.com/photos/372180634/medium.jpeg'

In [ ]:
!mkdir images

In [ ]:
#function for downloading images using requests lib
for url in tqdm(df['image_url']):
    response = requests.get(url)
    if response.status_code == 200:
        # saving image
        with open(f'images/{url.split("/")[-2]}', 'wb') as f:
            f.write(response.content)

In [ ]:
for filename in os.listdir(directory):
    # full path to file
    old_file = os.path.join(directory, filename)
    # make sure it's file
    if os.path.isfile(old_file):
        # new name with .jpg
        new_file = os.path.join(directory, f"{filename}.jpg")
        # rename
        os.rename(old_file, new_file)

In [ ]:
#checking for missing files

files = [f.split('.')[0] for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# set for faster search
file_set = set(files)

# extracting numbers from url
def extract_number(url):
    parts = url.split('/')
    return parts[-2]


df['image_number'] = df['image_url'].apply(extract_number)

# make another set
image_numbers_set = set(df['image_number'])

# calculate the difference
missing_files = image_numbers_set - file_set

print("Missing files:", missing_files)

In [ ]:
# checking for duplicates
value_counts = df['image_number'].value_counts()

duplicates = value_counts[value_counts > 1]
print(duplicates)


In [ ]:
# extracting only species with >99 observations

unique_names = df['scientific_name'].value_counts()

acceptable = unique_names[unique_names > 99]
print(acceptable)

In [ ]:
# dictionary
selected_mushrooms = ['Xanthoria parietina',
'Fomitopsis pinicola',
'Amanita muscaria',
'Fomes fomentarius',
'Hypogymnia physodes',
'Parmelia sulcata',
'Sarcoscypha austriaca',
'Platismatia glauca',
'Coprinus comatus',
'Laetiporus sulphureus',
'Fomitopsis betulina',
'Pseudevernia furfuracea',
'Boletus edulis',
'Evernia prunastri',
'Cantharellus cibarius',
'Leccinum scabrum',
'Vulpicida pinastri',
'Cetraria islandica',
'Flammulina',
'Ganoderma applanatum',
'Inonotus obliquus']

In [ ]:
for name in selected_mushrooms:
    # folders for each type
    dir_path = os.path.join(directory, name)
    os.makedirs(dir_path, exist_ok=True)

In [ ]:
# fill the dictionary
mushroom_photos = {}

for mushroom in selected_mushrooms:
    filtered_df = df[df['scientific_name'] == mushroom]
    mushroom_photos[mushroom] = filtered_df['image_number'].tolist()

In [ ]:
# fill the folders

for mushroom, photo_ids in mushroom_photos.items():
    mushroom_path = os.path.join(directory, mushroom)
    if not os.path.exists(mushroom_path):
        os.makedirs(mushroom_path)


    for photo_id in photo_ids:
        # adding .jpg
        filename = f"{photo_id}.jpg"
        # full path
        source_path = os.path.join(directory, filename)
        # full final path
        destination_path = os.path.join(mushroom_path, filename)


        if os.path.exists(source_path):
            shutil.move(source_path, destination_path)
        else:
            print(f"Файл {source_path} не найден.")